TODO list:

- wire all 3 servos reliably
- test communication with all servos at once
- create mounts on servo arms (small metal piece)

In [77]:
import serial
import numpy as np
from collections import defaultdict
import time

In [151]:
class ServoSerial:
    def __init__(self, num_servos, port='/dev/ttyACM0', baudrate=115200, offset=0.0):
        self.offset = 0.05
        self.min_angle = -np.pi/2.
        self.max_angle = np.pi/2.
        
        self.num_servos = num_servos
        self.serial = serial.Serial(port, baudrate=baudrate)
        self.curr_angles = defaultdict(float)
        for i in xrange(self.num_servos):
            self.set_angle(i, self.curr_angles[i])
        
    def is_valid_servo_num(self, servo_num):
        is_valid = 0 <= servo_num <= self.num_servos
        if not is_valid:
            print('WARNING: servo_num {0} not in [{1}, {2})'.format(servo_num, 0, self.num_servos))
        return is_valid
    
    def is_valid_angle(self, angle):
        is_valid = self.min_angle <= angle <= self.max_angle
        if not is_valid:
            print('WARNING: angle {0} not in [{1},{2}]'.format(angle, self.min_angle, self.max_angle))
        return is_valid
    
    def get_angle(self, servo_num):
        if not self.is_valid_servo_num(servo_num):
            return
        
        return self.curr_angles[servo_num]
        
    def set_angle(self, servo_num, des_angle, speed=None, fs=100.):
        """
        :param servo_num: servo to command
        :param des_angle: desired angle (radians)
        :param speed: if None, go as fast as possible. Else rad/s
        :param fs: if speed is not None, command discretization
        """
        if not (self.is_valid_servo_num(servo_num) and self.is_valid_angle(des_angle)):
            return
        
        if speed is None:
            self.serial.write('{0}{1}\n'.format(servo_num, self.offset+des_angle))
        else:
            curr_angle = self.curr_angles[servo_num]
            sign = 1 if des_angle > curr_angle else -1
            angles = np.r_[curr_angle:des_angle:sign*speed/fs]
            for angle in angles:
                self.serial.write('{0}{1}\n'.format(servo_num, self.offset+angle))
                self.curr_angles[servo_num] = angle
                time.sleep(1/fs)
                
        self.curr_angles[servo_num] = des_angle
        
s = ServoSerial(1)

In [150]:
curr_angle = s.get_angle(0)
des_angle = 0.
speed = 0.5
expected_time = abs(des_angle - curr_angle)/speed

start = time.time()
s.set_angle(0, des_angle, speed=speed)
end = time.time()

print('Expected time: {0}'.format(expected_time))
print('Elapsed: {0}'.format(end - start))

Expected time: 1.0
Elapsed: 1.04867601395
